In [1]:
import torch
import os
import wandb
from IPython.display import Image, clear_output  # to display images
from hyperopt import fmin, tpe, hp, Trials, space_eval
from ultralytics import YOLO
#wandb_api_key = user_secrets.get_secret("wandb_key")
#wandb.login(key=wandb_api_key)
wandb.login(key="6227e72bb3b48dc12a4035210349fec6a2b3637b")

print(f"Setup complete. Using torch {torch.__version__} ({torch.cuda.get_device_properties(0).name if torch.cuda.is_available() else 'CPU'})")

wandb: Currently logged in as: edcogue (edcogue2207). Use `wandb login --relogin` to force relogin
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


Setup complete. Using torch 2.0.1+cu117 (NVIDIA GeForce GTX 1660 Ti)


In [2]:
def objective(params):
    # Define el modelo con los parámetros dados
    model = YOLO().load('yolov8n.pt')
    
    # Entrena el modelo
    model.train(data="/tf/data/Mammographies/yolo_data/data_object_detection.yaml",
                epochs=25,
                dropout=params["dropout"],
                lr0=params["lr"],                
                batch=2,
                augment=True,
                imgsz=1600)
    
    # Evalúa el modelo
    metrics = model.val()
    
    # Devuelve la métrica a optimizar
    return -metrics.results_dict["metrics/recall(B)"]

# Define los rangos de los hiperparámetros
space = {
    'dropout': hp.choice('dropout',[0.0, 0.2, 0.4]),
    'lr': hp.loguniform('lr', -6, -1),
}

trials = Trials()

# Realiza la búsqueda de hiperparámetros
best = fmin(objective, space, algo=tpe.suggest, max_evals=15, trials=trials)

  0%|          | 0/2 [00:00<?, ?trial/s, best loss=?]

Transferred 355/355 items from pretrained weights
New https://pypi.org/project/ultralytics/8.0.109 available 😃 Update with 'pip install -U ultralytics'
Ultralytics YOLOv8.0.98 🚀 Python-3.8.10 torch-2.0.1+cu117 CUDA:0 (NVIDIA GeForce GTX 1660 Ti, 5945MiB)
yolo/engine/trainer: task=detect, mode=train, model=yolov8n.pt, data=/tf/data/Mammographies/yolo_data/data_object_detection.yaml, epochs=1, patience=50, batch=1, imgsz=100, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=None, exist_ok=False, pretrained=False, optimizer=SGD, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=0, resume=False, amp=True, overlap_mask=True, mask_ratio=4, dropout=0.2, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, show=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True, show_conf=True, vid_stride=1, line_width=Non

AMP: running Automatic Mixed Precision (AMP) checks with YOLOv8n...
AMP: checks passed ✅
WARNING ⚠️ imgsz=[100] must be multiple of max stride 32, updating to [128]
optimizer: SGD(lr=0.006866187368150327) with parameter groups 57 weight(decay=0.0), 64 weight(decay=0.0005), 63 bias
train: Scanning /tf/data/Mammographies/yolo_data/object_detection/train/labels.cache... 2442 images, 0 backgrounds, 0 corrupt: 100%|##########| 2442/2442 [00:00<?, ?it/s]

train: Scanning /tf/data/Mammographies/yolo_data/object_detection/train/labels.cache... 2442 images, 0 backgrounds, 0 corrupt: 100%|##########| 2442/2442 [00:00<?, ?it/s]
albumentations: Blur(p=0.01, blur_limit=(3, 7)), MedianBlur(p=0.01, blur_limit=(3, 7)), ToGray(p=0.01), CLAHE(p=0.01, clip_limit=(1, 4.0), tile_grid_size=(8, 8))
val: Scanning /tf/data/Mammographies/yolo_data/object_detection/val/labels.cache... 695 images, 0 backgrounds, 0 corrupt: 100%|##########| 695/695 [00:00<?, ?it/s]

val: Scanning /tf/data/Mammographies/yolo_data/o


        1/1     0.138G     0.5216     0.6311     0.2358          2        128:   1%|1         | 29/2442 [00:03<04:11,  9.58it/s]

        1/1     0.138G     0.5042     0.6149     0.2279          3        128:   1%|1         | 29/2442 [00:03<04:11,  9.58it/s]

        1/1     0.138G     0.5042     0.6149     0.2279          3        128:   1%|1         | 30/2442 [00:03<04:10,  9.63it/s]

        1/1     0.138G     0.4879     0.5996     0.2206          0        128:   1%|1         | 30/2442 [00:03<04:10,  9.63it/s]

        1/1     0.138G     0.4727     0.5852     0.2137          0        128:   1%|1         | 30/2442 [00:03<04:10,  9.63it/s]

        1/1     0.138G     0.4727     0.5852     0.2137          0        128:   1%|1         | 32/2442 [00:03<04:01,  9.99it/s]

        1/1     0.138G     0.4584     0.5716     0.2072          1        128:   1%|1         | 32/2442 [00:03<04:01,  9.99it/s]

        1/1     0.138G     0.4584     0.5716     0.2072          1        128:   1%|1    


        1/1     0.138G     0.5492     0.6649     0.2671          3        128:   3%|2         | 66/2442 [00:07<03:50, 10.31it/s]

        1/1     0.138G     0.5411     0.6563     0.2632          3        128:   3%|2         | 66/2442 [00:07<03:50, 10.31it/s]

        1/1     0.138G     0.5411     0.6563     0.2632          3        128:   3%|2         | 68/2442 [00:07<04:15,  9.28it/s]

        1/1     0.138G     0.5332     0.6481     0.2594          0        128:   3%|2         | 68/2442 [00:07<04:15,  9.28it/s]

        1/1     0.138G     0.5332     0.6481     0.2594          0        128:   3%|2         | 69/2442 [00:07<04:27,  8.86it/s]

        1/1     0.138G     0.5256       0.64     0.2557          2        128:   3%|2         | 69/2442 [00:07<04:27,  8.86it/s]

        1/1     0.138G     0.5256       0.64     0.2557          2        128:   3%|2         | 70/2442 [00:07<04:37,  8.54it/s]

        1/1     0.138G     0.5357     0.6433     0.2654          1        128:   3%|2    

...


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95):  67%|######6   | 233/348 [00:05<00:02, 41.67it/s]

                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95):  68%|######8   | 238/348 [00:05<00:02, 43.03it/s]

                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95):  70%|######9   | 243/348 [00:05<00:02, 41.12it/s]

                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95):  71%|#######1  | 248/348 [00:05<00:02, 42.75it/s]

                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95):  73%|#######2  | 253/348 [00:05<00:02, 41.95it/s]

                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95):  74%|#######4  | 258/348 [00:06<00:02, 42.81it/s]

                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95):  76%|#######5  | 263/348 [00:06<00:02, 41.45it/s]

            


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95):  15%|#5        | 106/695 [00:03<00:17, 33.15it/s]

                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95):  16%|#5        | 110/695 [00:03<00:17, 33.41it/s]

                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95):  16%|#6        | 114/695 [00:03<00:17, 33.26it/s]

                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95):  17%|#6        | 118/695 [00:03<00:17, 33.29it/s]

                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95):  18%|#7        | 122/695 [00:03<00:17, 33.28it/s]

                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95):  18%|#8        | 126/695 [00:03<00:17, 33.29it/s]

                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95):  19%|#8        | 130/695 [00:03<00:17, 32.79it/s]

            


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95):  48%|####8     | 334/695 [00:10<00:10, 33.68it/s]

                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95):  49%|####8     | 338/695 [00:10<00:10, 33.58it/s]

                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95):  49%|####9     | 342/695 [00:10<00:10, 33.48it/s]

                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95):  50%|####9     | 346/695 [00:10<00:10, 33.66it/s]

                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95):  50%|#####     | 350/695 [00:10<00:10, 34.00it/s]

                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95):  51%|#####     | 354/695 [00:10<00:09, 34.32it/s]

                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95):  52%|#####1    | 358/695 [00:10<00:09, 34.04it/s]

            


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95):  81%|########  | 562/695 [00:17<00:04, 33.14it/s]

                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95):  81%|########1 | 566/695 [00:17<00:03, 32.84it/s]

                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95):  82%|########2 | 570/695 [00:17<00:03, 32.60it/s]

                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95):  83%|########2 | 574/695 [00:17<00:03, 32.47it/s]

                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95):  83%|########3 | 578/695 [00:17<00:03, 32.76it/s]

                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95):  84%|########3 | 582/695 [00:17<00:03, 32.77it/s]

                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95):  84%|########4 | 586/695 [00:17<00:03, 32.27it/s]

            

100%|██████████| 2/2 [09:20<00:00, 280.17s/trial, best loss: -0.06467661691542288]
{'dropout': 1, 'lr': 0.006866187368150327}
[{'state': 2, 'tid': 0, 'spec': None, 'result': {'loss': -0.06467661691542288, 'status': 'ok'}, 'misc': {'tid': 0, 'cmd': ('domain_attachment', 'FMinIter_Domain'), 'workdir': None, 'idxs': {'dropout': [0], 'lr': [0]}, 'vals': {'dropout': [1], 'lr': [0.006866187368150327]}}, 'exp_key': None, 'owner': None, 'version': 0, 'book_time': datetime.datetime(2023, 5, 26, 22, 12, 22, 117000), 'refresh_time': datetime.datetime(2023, 5, 26, 22, 17, 2, 264000)}, {'state': 2, 'tid': 1, 'spec': None, 'result': {'loss': -0.05472636815920398, 'status': 'ok'}, 'misc': {'tid': 1, 'cmd': ('domain_attachment', 'FMinIter_Domain'), 'workdir': None, 'idxs': {'dropout': [1], 'lr': [1]}, 'vals': {'dropout': [2], 'lr': [0.0032369011039803082]}}, 'exp_key': None, 'owner': None, 'version': 0, 'book_time': datetime.datetime(2023, 5, 26, 22, 17, 2, 267000), 'refresh_time': datetime.datetime(2

In [5]:
trials.trials

[{'state': 2,
  'tid': 0,
  'spec': None,
  'result': {'loss': -0.06467661691542288, 'status': 'ok'},
  'misc': {'tid': 0,
   'cmd': ('domain_attachment', 'FMinIter_Domain'),
   'workdir': None,
   'idxs': {'dropout': [0], 'lr': [0]},
   'vals': {'dropout': [1], 'lr': [0.006866187368150327]}},
  'exp_key': None,
  'owner': None,
  'version': 0,
  'book_time': datetime.datetime(2023, 5, 26, 22, 12, 22, 117000),
  'refresh_time': datetime.datetime(2023, 5, 26, 22, 17, 2, 264000)},
 {'state': 2,
  'tid': 1,
  'spec': None,
  'result': {'loss': -0.05472636815920398, 'status': 'ok'},
  'misc': {'tid': 1,
   'cmd': ('domain_attachment', 'FMinIter_Domain'),
   'workdir': None,
   'idxs': {'dropout': [1], 'lr': [1]},
   'vals': {'dropout': [2], 'lr': [0.0032369011039803082]}},
  'exp_key': None,
  'owner': None,
  'version': 0,
  'book_time': datetime.datetime(2023, 5, 26, 22, 17, 2, 267000),
  'refresh_time': datetime.datetime(2023, 5, 26, 22, 21, 42, 447000)}]

In [16]:
best = space_eval(space, best)

In [17]:
best

{'dropout': 0.2, 'lr': 0.006866187368150327}

In [24]:
# Define el modelo con los parámetros dados
model = YOLO().load('yolov8n.pt')

# Entrena el modelo
model.train(data="/tf/data/Mammographies/yolo_data/data_object_detection.yaml",
            epochs=100,
            dropout=best["dropout"],
            lr0=best["lr"],                
            batch=2,
            augment=True,
            imgsz=1600)    

Transferred 355/355 items from pretrained weights
New https://pypi.org/project/ultralytics/8.0.109 available 😃 Update with 'pip install -U ultralytics'
Ultralytics YOLOv8.0.98 🚀 Python-3.8.10 torch-2.0.1+cu117 CUDA:0 (NVIDIA GeForce GTX 1660 Ti, 5945MiB)
yolo/engine/trainer: task=detect, mode=train, model=yolov8n.pt, data=/tf/data/Mammographies/yolo_data/data_object_detection.yaml, epochs=1, patience=50, batch=1, imgsz=100, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=None, exist_ok=False, pretrained=False, optimizer=SGD, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=0, resume=False, amp=True, overlap_mask=True, mask_ratio=4, dropout=0.2, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, show=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True, show_conf=True, vid_stride=1, line_width=Non

In [25]:
# Evalúa el modelo
metrics = model.val()

Ultralytics YOLOv8.0.98 🚀 Python-3.8.10 torch-2.0.1+cu117 CUDA:0 (NVIDIA GeForce GTX 1660 Ti, 5945MiB)
Model summary (fused): 168 layers, 3005843 parameters, 0 gradients, 8.1 GFLOPs
val: Scanning /tf/data/Mammographies/yolo_data/object_detection/val/labels.cache... 695 images, 0 backgrounds, 0 corrupt: 100%|██████████| 695/695 [00:00<?, ?it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 695/695 [00:21<00:00, 32.52it/s]
                   all        695       2211    0.00517     0.0647    0.00302   0.000801
Speed: 0.3ms preprocess, 4.8ms inference, 0.0ms loss, 0.9ms postprocess per image
Results saved to runs/detect/val


In [26]:
metrics

ultralytics.yolo.utils.metrics.DetMetrics object with attributes:

ap_class_index: array([0])
box: ultralytics.yolo.utils.metrics.Metric object
confusion_matrix: <ultralytics.yolo.utils.metrics.ConfusionMatrix object at 0x7fe0c47aae20>
fitness: 0.0010233899347328534
keys: ['metrics/precision(B)', 'metrics/recall(B)', 'metrics/mAP50(B)', 'metrics/mAP50-95(B)']
maps: array([ 0.00080105])
names: {0: 'Abnormalities'}
plot: True
results_dict: {'metrics/precision(B)': 0.005170294309060669, 'metrics/recall(B)': 0.06467661691542288, 'metrics/mAP50(B)': 0.0030244826828558146, 'metrics/mAP50-95(B)': 0.0008010462960525242, 'fitness': 0.0010233899347328534}
save_dir: PosixPath('runs/detect/val')
speed: {'preprocess': 0.29190358498113617, 'inference': 4.80967871576762, 'loss': 0.010422836962363702, 'postprocess': 0.8912940677121389}

In [28]:
model.export()

Ultralytics YOLOv8.0.98 🚀 Python-3.8.10 torch-2.0.1+cu117 CPU

PyTorch: starting from runs/detect/train2/weights/best.pt with input shape (1, 3, 128, 128) BCHW and output shape(s) (1, 5, 336) (5.9 MB)

TorchScript: starting export with torch 2.0.1+cu117...
TorchScript: export success ✅ 0.9s, saved as runs/detect/train2/weights/best.torchscript (11.8 MB)

Export complete (1.0s)
Results saved to /tf/code/YOLOv8/object_detector/runs/detect/train2/weights
Predict:         yolo predict task=detect model=runs/detect/train2/weights/best.torchscript imgsz=128 
Validate:        yolo val task=detect model=runs/detect/train2/weights/best.torchscript imgsz=128 data=/tf/data/Mammographies/yolo_data/data_object_detection.yaml 
Visualize:       https://netron.app


'runs/detect/train2/weights/best.torchscript'

In [10]:
model = YOLO('./runs/detect/train2/weights/best.pt')
results=model("test.png",save=True) #Saved on runs/detect/predict
results


image 1/1 /tf/code/YOLOv8/object_detector/test.png: 128x128 (no detections), 3.9ms
Speed: 0.5ms preprocess, 3.9ms inference, 0.4ms postprocess per image at shape (1, 3, 128, 128)
Results saved to runs/detect/predict9


[ultralytics.yolo.engine.results.Results object with attributes:
 
 boxes: ultralytics.yolo.engine.results.Boxes object
 keypoints: None
 keys: ['boxes']
 masks: None
 names: {0: 'Abnormalities'}
 orig_img: array([[[13, 13, 13],
         [13, 13, 13],
         [13, 13, 13],
         ...,
         [13, 13, 13],
         [13, 13, 13],
         [13, 13, 13]],
 
        [[13, 13, 13],
         [13, 13, 13],
         [13, 13, 13],
         ...,
         [13, 13, 13],
         [13, 13, 13],
         [13, 13, 13]],
 
        [[13, 13, 13],
         [13, 13, 13],
         [13, 13, 13],
         ...,
         [13, 13, 13],
         [13, 13, 13],
         [13, 13, 13]],
 
        ...,
 
        [[13, 13, 13],
         [13, 13, 13],
         [13, 13, 13],
         ...,
         [13, 13, 13],
         [13, 13, 13],
         [13, 13, 13]],
 
        [[13, 13, 13],
         [13, 13, 13],
         [13, 13, 13],
         ...,
         [13, 13, 13],
         [13, 13, 13],
         [13, 13, 13]],
 
    